In [ ]:
import subprocess
import shlex
import re
import json

import numpy as np
import multiprocessing as mp

from matplotlib import pyplot as plt
from scipy import stats
from tqdm import tqdm
from IPython.display import display, Markdown
from tempfile import TemporaryDirectory

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

def show():
    plt.savefig(f"{show.fig_counter}.pdf", dpi='figure', transparent=True, bbox_inches='tight')
    show.fig_counter += 1
    show.plt_show()

show.fig_counter = 0
show.plt_show = plt.show

plt.show = show

In [ ]:
EVENT_PATTERN = re.compile(r"Event Summary:\n - Total #individuals:\n   \d+\n - Total #events:\n   - raw:\n     (\d+)\n   - deduplicated:\n     (\d+)")
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")

In [ ]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

In [ ]:
def simulate_throughput_monolithic(algorithm, speciation=0.001, seed=42, sample=1.0, scenario="NonSpatial(area:(100,100),deme:100)"):
    config = "".join(f"""
    (
        speciation: {speciation},
        sample: {sample},
        seed: {seed},

        algorithm: {algorithm},

        log: None,

        scenario: {scenario},

        reporters: [
            Plugin(
                library: "{target_directory}/release/deps/libnecsim_plugins_common.so",
                reporters: [Counter(), Execution()]
            )
        ],
    )
    """.split()).replace(",)", ")").replace(",]", "]")

    # Run the simulation
    result = subprocess.run(shlex.split(
        f"{target_directory}/release/rustcoalescence simulate '{config}'"
        #"cargo run --release --features rustcoalescence-algorithms-monolithic,"
        #+ "rustcoalescence-algorithms-independent,necsim-partitioning-mpi "
        #+ f"--quiet -- simulate '{config}'"
    ), check=True, capture_output=True, text=True)

    match = EVENT_PATTERN.search(result.stdout)
    if match is None:
        print(result.stdout)
        print(result.stderr)
    raw_events = int(match.group(1))
    deduplicated_events = int(match.group(2))
    
    match = EXECUTION_PATTERN.search(result.stdout)
    if match is None:
        print(result.stdout)
        print(result.stderr)
    initialisation = parse_time(match.group(1))
    execution = parse_time(match.group(2))
    cleanup = parse_time(match.group(3))
        
    return raw_events, deduplicated_events, initialisation, execution, cleanup

In [ ]:
def batch_simulation_many_seeds(simulate, seeds, args=tuple(), kwargs=dict(), silent=False, processes=mp.cpu_count()):
    results = []

    with tqdm(total=len(seeds), disable=silent) as progress:
        def update_progress(result):
            results.append(result)

            progress.update()
        
        def update_error(err):
            print(err)

        with mp.Pool(processes) as pool:
            for seed in seeds:
                pool.apply_async(simulate, args, {**kwargs, "seed": seed}, update_progress, update_error)

            pool.close()
            pool.join()
    
    return results

In [ ]:
display(Markdown("# Event throughput"))

for algorithm in [
    "Classical()", "Gillespie()", "SkippingGillespie()",
    f"""Independent(
        dedup_cache: Relative(factor: {1.0}),
        delta_t: {2.0},
        parallelism_mode: Monolithic(event_slice: {100*100*100})
    )"""
]:
    display(Markdown(f"## {algorithm[:algorithm.find('(')]}:"))
    
    for scenario, sample in [
        (f"NonSpatial(area: ({100}, {100}), deme: {100})", 1.0),
        (f"AlmostInfinite(radius: {564}, sigma: {10.0})", 1.0),
        (f"""SpatiallyExplicit(
            habitat: "{target_directory}/../maps/madingley/fg0size12/habitat.tif",
            dispersal: "{target_directory}/../maps/madingley/fg0size12/dispersal.tif"
        )""", 0.000025),
    ]:
        seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=160)

        raw_events, deduplicated_events, initialisations, executions, cleanups = zip(*batch_simulation_many_seeds(
            simulate_throughput_monolithic, seeds, args=(algorithm,), kwargs={
                "scenario":"NonSpatial(area:(100,100),deme:100)", "speciation":0.001
            }, silent=False
        ))
        
        raw_throughput = np.mean(raw_events) / np.mean(executions)
        throughput = np.mean(deduplicated_events) / np.mean(executions)
        
        if raw_throughput != throughput:
            display(Markdown(f"### {scenario[:scenario.find('(')]}: {np.round(throughput, 1)}/s [raw: {np.round(raw_throughput, 1)}/s]"))
        else:
            display(Markdown(f"### {scenario[:scenario.find('(')]}: {np.round(throughput, 1)}/s"))